In [1]:
#!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


     |████████████████████████████████| 993kB 1.4MB/s 


In [2]:
# Library loading
%matplotlib inline

import pandas as pd # manipulate dataframes
import matplotlib.pyplot as plt # plotting
import matplotlib
import numpy as np
import time

from sklearn.metrics import mean_squared_error
import h5py

# Check torch install
try:
  import torch
except:
  print("Starting a session, torch not installed, installing...")
  !pip3 install torch # we install torch if not installed
  import torch

# First we check if CUDA is available
print("CUDA AVAILABLE? ",torch.cuda.is_available())

def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
      
device = get_default_device()
print(device)

CUDA AVAILABLE?  True
cuda


In [3]:
### Download datasets
downloaded = drive.CreateFile({'id':"1t5wi4xjrOD5OcxH3SMhTNH9wxRuU5dMb"})   # replace the id with id of file you want to access
downloaded.GetContentFile('NKAS_Raman.hdf5')  

downloaded = drive.CreateFile({'id':"1wq_e-gcfHjf3Wes6-Vsur-9zNKfhgo-9"})   # replace the id with id of file you want to access
downloaded.GetContentFile('DataSets.zip') 

!unzip DataSets.zip

downloaded = drive.CreateFile({'id':"1FxLvyBgmfQ17xctfOjzEvWcNmGa9F4sR"})   # replace the id with id of file you want to access
downloaded.GetContentFile('NKAS_density.hdf5')  

downloaded = drive.CreateFile({'id':"1AyFwtkEzhH01clvoo9Y5_unGfeMM8Q1E"})   # replace the id with id of file you want to access
downloaded.GetContentFile('neuravi.py') 

Archive:  DataSets.zip
  inflating: DataSet_0p80val.hdf5    
  inflating: DataSet_0p20val.hdf5    
  inflating: DataSet_0p30val.hdf5    
  inflating: DataSet_0p50val.hdf5    
  inflating: DataSet_0p40val.hdf5    
  inflating: DataSet_0p10val.hdf5    
  inflating: DataSet_0p60val.hdf5    
  inflating: DataSet_0p70val.hdf5    


In [0]:
import neuravi

def train_model(path_data,nb_neurons,nb_layers,p_drop, name, device):

    ds = neuravi.data_loader(path_data,"./NKAS_Raman.hdf5","./NKAS_density.hdf5",device)
    
    # declaring model
    neuralmodel = neuravi.model(4,nb_neurons,nb_layers,ds.nb_channels_raman,p_drop=p_drop) 

    # criterion for match
    criterion = torch.nn.MSELoss()
    criterion.to(device)
    optimizer = torch.optim.Adam(neuralmodel.parameters(), lr = 0.001) # optimizer

    # we initialize the output bias
    neuralmodel.output_bias_init()

    # we send the neural net on device
    neuralmodel.to(device)
    
    #
    # PRETRAINING
    #
    neuralmodel, record_pretrain_loss, record_prevalid_loss = neuravi.pretraining(neuralmodel,ds,criterion,optimizer,verbose=False)
                
    #
    # TRAINING
    #
    neuralmodel, record_train_loss, record_valid_loss = neuravi.maintraining(neuralmodel,ds,criterion,optimizer,name,verbose=False)
        
    #
    # SAVING MODEL
    #
    try:
        file_out = drive.CreateFile()
        # Read file and set it as a content of this instance.
        file_out.SetContentFile(name)
        file_out.Upload() # Upload the file.
        print('Model {} saved'.format(name))
    except:
        print("Warning, training never converged for dataset {}, model not saved".format(path_data))

In [5]:
path_data = ["./DataSet_0p10val.hdf5",
             "./DataSet_0p20val.hdf5",
             "./DataSet_0p30val.hdf5",
             "./DataSet_0p40val.hdf5",
             "./DataSet_0p50val.hdf5",
             "./DataSet_0p60val.hdf5",
             "./DataSet_0p70val.hdf5",
             "./DataSet_0p80val.hdf5"]
save_names = ["model_l4_n200_p20_data0p10val",
              "model_l4_n200_p20_data0p20val",
              "model_l4_n200_p20_data0p30val",
              "model_l4_n200_p20_data0p40val",
              "model_l4_n200_p20_data0p50val",
              "model_l4_n200_p20_data0p60val",
              "model_l4_n200_p20_data0p70val",
              "model_l4_n200_p20_data0p80val"]

nb_neurons = 200
nb_layers = 4
p_drop = 0.20

for i in range(len(path_data)):
    print('Experiment on dataset {} started...'.format(i))
    for j in range(10):
        train_model(path_data[i], nb_neurons, nb_layers, p_drop, 
                    save_names[i]+"_{}.pth".format(j), device)

Experiment on dataset 0 started...
Model model_l4_n200_p20_data0p10val_0.pth saved
Model model_l4_n200_p20_data0p10val_1.pth saved
Model model_l4_n200_p20_data0p10val_2.pth saved
Model model_l4_n200_p20_data0p10val_3.pth saved
Model model_l4_n200_p20_data0p10val_4.pth saved
Model model_l4_n200_p20_data0p10val_5.pth saved
Model model_l4_n200_p20_data0p10val_6.pth saved
Model model_l4_n200_p20_data0p10val_7.pth saved
Model model_l4_n200_p20_data0p10val_8.pth saved
Model model_l4_n200_p20_data0p10val_9.pth saved
Experiment on dataset 1 started...
Model model_l4_n200_p20_data0p20val_0.pth saved
Model model_l4_n200_p20_data0p20val_1.pth saved
Model model_l4_n200_p20_data0p20val_2.pth saved
Model model_l4_n200_p20_data0p20val_3.pth saved
Model model_l4_n200_p20_data0p20val_4.pth saved
Model model_l4_n200_p20_data0p20val_5.pth saved
Model model_l4_n200_p20_data0p20val_6.pth saved
Model model_l4_n200_p20_data0p20val_7.pth saved
Model model_l4_n200_p20_data0p20val_8.pth saved
Model model_l4_n20

In [0]:
Goo